In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
import tensorflow as tf
import tflearn
import numpy as np
import matplotlib.pyplot as plt

from data_prepare import mnist
from model import inference
from train import train_op
mnist = mnist()

In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32,[None, 28,28,1])
y = tf.placeholder(tf.int64,[None])
ratio_s = 1
ratio_c = 0
ratio_t = 0
ratio_g = 0
logits, features = inference(x)
embeddings = tf.nn.l2_normalize(features, axis=1)
trainer, loss = train_op(y, logits, features, embeddings, ratio_s=ratio_s, ratio_c=ratio_c, 
                         ratio_t=ratio_t, ratio_g=ratio_g)

with tf.name_scope('acc'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), y), tf.float32))

mean_data = np.mean(mnist.train.images, axis=0)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
step = 0
tflearn.is_training(True, session=sess)
while step <= 20000:
    batch_images, batch_labels = mnist.train.next_batch(128)
    _, train_acc, train_loss = sess.run([trainer, accuracy, loss], feed_dict={x: batch_images - mean_data, y: batch_labels})
    step += 1
    
    if step % 1000 == 0:
        tflearn.is_training(False, session=sess)
        vali_image = mnist.validation.images - mean_data
        vali_acc = sess.run(accuracy, feed_dict={x: vali_image, y: mnist.validation.labels})
        print(("step: {}, train_acc:{:.4f}, train_loss:{:.4f}, vali_acc:{:.4f}".
               format(step, train_acc, train_loss, vali_acc)))
        tflearn.is_training(True, session=sess)
        
tflearn.is_training(False, session=sess)

In [ ]:
%matplotlib inline

f = plt.figure(figsize=(18,18))
c = ['#ff0000', '#ffff00', '#00ff00', '#00ffff', '#0000ff', 
     '#ff00ff', '#990000', '#999900', '#009900', '#009999']

feat, emb = sess.run([features, embeddings], feed_dict={x:mnist.train.images[:10000]-mean_data})
labels = mnist.train.labels[:10000]
plt.subplot(2,2,1)
for i in range(10):
    plt.plot(feat[labels==i,0].flatten(), feat[labels==i,1].flatten(), '.', c=c[i])
plt.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
plt.title('train feature')
plt.grid()

plt.subplot(2,2,3)
for i in range(10):
    plt.plot(emb[labels==i,0].flatten(), emb[labels==i,1].flatten(), '.', c=c[i])
plt.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
plt.title('train embedding')
plt.grid()

feat, emb = sess.run([features, embeddings], feed_dict={x:mnist.test.images[:10000]-mean_data})
labels = mnist.test.labels[:10000]
plt.subplot(2,2,2)
for i in range(10):
    plt.plot(feat[labels==i,0].flatten(), feat[labels==i,1].flatten(), '.', c=c[i])
plt.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
plt.title('valid feature')
plt.grid()

plt.subplot(2,2,4)
for i in range(10):
    plt.plot(emb[labels==i,0].flatten(), emb[labels==i,1].flatten(), '.', c=c[i])
plt.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
plt.title('valid embedding')
plt.grid()

plt.show()